In [2]:
import os
import pytesseract
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

In [4]:
import os
import fitz
from PIL import Image
from comtypes import client
import shutil


'''
Очистка данных перед обучением нейросети.
'''

INPUT_FOLDER = "Материалы ИИ ХАКАТОН"


# Переименование всех папок в формат 1..41
def rename_folders_and_files(root_directory):
    for folder_name in os.listdir(root_directory):
        if os.path.isdir(os.path.join(root_directory, folder_name)):
            old_folder_path = os.path.join(root_directory, folder_name)
            new_folder_name = str(int(folder_name.split()[1]))
            new_folder_path = os.path.join(root_directory, new_folder_name)
            os.rename(old_folder_path, new_folder_path)

            # Переименование файлов внутри каждой папки
            for file_name in os.listdir(new_folder_path):
                file_path = os.path.join(new_folder_path, file_name)
                if file_name.lower().endswith('.pdf'):
                    # Название для PDF файлов
                    if "договор аренды" in file_name.lower() or "договор" in file_name.lower():
                        new_file_name = "договор.pdf"
                    elif "расчет" in file_name.lower():
                        new_file_name = "расчёт.pdf"
                    elif "претензия" in file_name.lower():
                        new_file_name = "претензия.pdf"
                    else:
                        new_file_name = file_name
                    os.rename(file_path, os.path.join(new_folder_path, new_file_name.lower()))

                elif file_name.lower().endswith('.docx'):
                    new_docx_name = f"{new_folder_name}.docx"  # Название должно быть равно номеру папки
                    os.rename(file_path, os.path.join(new_folder_path, new_docx_name))
                elif file_name.lower().endswith('.doc'):
                    shutil.rmtree(new_folder_path)
                    break


rename_folders_and_files(INPUT_FOLDER)


# Конвертация всех пдфок в картинки
def pdf_to_images(pdf_path, output_directory):
    pdf_document = fitz.open(pdf_path)

    for page_number in range(pdf_document.page_count):
        page = pdf_document.load_page(page_number)
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        img.save(os.path.join(output_directory, f"page_{page_number + 1}.jpg"))  # Сохранение изображения


def convert_pdfs_to_images(root_directory):
    for folder_name in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                if file_name.lower().endswith('.pdf'):
                    pdf_document_type = file_name.split('.')[0]
                    output_directory = os.path.join(folder_path, pdf_document_type)
                    os.makedirs(output_directory, exist_ok=True)
                    pdf_to_images(file_path, output_directory)


convert_pdfs_to_images(INPUT_FOLDER)

In [5]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_from_images(folder_path):
    texts = []
    labels = []
    court_types = {1: 1, 10: 0, 13: 1, 17: 1, 2: 0, 23: 1, 28: 0, 3: 0, 31: 0, 33: 0, 34: 0, 35: 1, 37: 0, 38: 1, 4: 1, 41: 0, 5: 0, 6: 0, 7: 1, 8: 1, 9: 1}
    for subdir in os.listdir(folder_path):
        subdir_path = os.path.join(data_folder, subdir)
        label = court_types[int(subdir)]
        labels.append(label)
        if os.path.isdir(subdir_path):
            text = ""
            for subsubdir in os.listdir(subdir_path):
                subsubdir_path = os.path.join(subdir_path, subsubdir)
                if os.path.isdir(subsubdir_path):
                    if subsubdir.lower() == 'договор' or subsubdir.lower() == 'претензия':
                        for file_name in os.listdir(subsubdir_path):
                            file_path = os.path.join(subsubdir_path, file_name)
                            text += pytesseract.image_to_string(file_path, lang='rus')
            texts.append(text)
    return texts, labels

data_folder = 'Материалы ИИ ХАКАТОН'
texts, labels = extract_text_from_images(data_folder)

In [6]:
print(texts)

['=> >\n\nдоговор № ЧИ\n\nаренды земельного участка\n\nг.Тула >, ЧЕ «=>.\n\nМинистерство нмущественных и земельных отношений Тульской\nобласти, именуемое в дальнейшем АРЕНДОДАТЕЛЬ, в лице министра\n`Шерина Валерия Витальевича, действующего на основании Положения, с\nодной стороны, с одной стороны, и\n\nзыскуемый в дальнейшем «Арендатор», © другой стороны, на освованкя\nпротокола полвелелия итогов яукшиона открытого по составу участяиков и\nпо форме полачи предложений по пролаже права на заключение договора\nаренды земельного участка от ФВИИШИЩЬ, заключили настоящий договор (в\nдальнейшем — «логовор») о нижеследующем.\n\n1. ПРЕДМЕТ ДОГОВОРА.\n\n11. Ароилолатель прелоставаяет, а Арендатор принимает в аренду\nэвельный участок:\nкатегория Земель земли населенных пунктов,\n\nхазастровый вомер ФВ,\nрасположенный адресу\n\nименуемый в дальнейшем «участок»,\n\nдля строительства отдельно стоящего индивидуального жилого лома в 1-3\nэтажа (разрешенное испольювание — отдельно стоящий индивидуальны

In [7]:
# векторизация
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model.fit(X_train.toarray(), y_train, epochs=100, batch_size=32, validation_data=(X_test.toarray(), y_test))

# Проверка производительности модели
loss, accuracy = model.evaluate(X_test.toarray(), y_test)
print(f"Test Accuracy: {accuracy}")



Epoch 1/100


1/1 [==============================] - 4s 4s/step - loss: 1.1289 - accuracy: 0.4375 - val_loss: 0.7003 - val_accuracy: 0.8000
Epoch 2/100
1/1 [==============================] - 0s 141ms/step - loss: 1.4023 - accuracy: 0.5625 - val_loss: 1.1300 - val_accuracy: 0.4000
Epoch 3/100
1/1 [==============================] - 0s 44ms/step - loss: 0.7753 - accuracy: 0.6250 - val_loss: 1.3234 - val_accuracy: 0.4000
Epoch 4/100
1/1 [==============================] - 0s 46ms/step - loss: 0.7060 - accuracy: 0.7500 - val_loss: 1.0259 - val_accuracy: 0.4000
Epoch 5/100
1/1 [==============================] - 0s 46ms/step - loss: 0.6091 - accuracy: 0.6875 - val_loss: 1.1119 - val_accuracy: 0.4000
Epoch 6/100
1/1 [==============================] - 0s 47ms/step - loss: 0.4279 - accuracy: 0.8125 - val_loss: 1.2900 - val_accuracy: 0.4000
Epoch 7/100
1/1 [==============================] - 0s 53ms/step - loss: 0.2775 - accuracy: 0.8750 - val_loss: 1.3640 - val_accuracy: 0.4000
Epoch 8/100
1/1 [=

In [8]:
# Договор и претензия, не участвовашие в обучении:
text = """ 
I >ЧЕЖДЕНПЕ
Регаст
Д О Г О В О Р
АРЕНДЫ ЗЕМ ЕЛЬ НОГО УЧАСТКА
( Кадастровый М
Ту;га
ЕУ1униципальное (унитарное) предприятие «Тульский городской центр градостроительства и землеустройства» (6 ДачънеГнаем - АРЕНДОДА ТЕМЬ), действующее в общественных (муниципальных) интересах на основании Устава и в соответствии с делегированными постановлением Главы админисграции гТулы Уд 482 от 06.05. 1997г. правами, в лице директо а ЗАЙВOГ0 ВИТАЛИЯ ВИКТОРОВИЧА, с одной стороны, и
АРЕНДА ТОР), действующее на основании Устава, Б лице директора с другой стороны, на основании постановления Главы администрации Центрального района г. Тулы от заключили настоящий договор (в дальнейшем — ДОГОВОР) о нижеследующем
1.ПРЕДМЕТ ДОГОВОРА.
1 АРЕНДОДАТЕЈЉ предоставляет. а АРЕНДАТОР инимает в аренду земельный ччасток	площадью	КВ. М.	расположенный	в
(в с)азьнейшем -
1	2. раницы УЧАСТКА закреплены в натуре и обозначены на прилагаемом к
ОГОВОРУ плане земельного участка поворотными точками
2. РАЗРЕШЁННОЕ ИСПОЛЬЗОВАНИЕ УЧАСТКА.
2	1. УЧАСТОК предоставляется для организации входа в выставочный зал (в ()ачьнейигем - ОБЪЕКТ).
З. СРОК двйствия ДОГОВОРА,
З. ! настоящий ДОГОВОР заключён сроком до
3.2 Продление срока действия (пролонгация) ДОГОВОРА оформляется соглашением, оторое заключается сторонами на основании соответствующего постановления дминистрации Центрального района г. Тулы.
З З О нродлении (пролонгации) ДОГОВОРА, АРЕНДОДАТЕЛЬ информирует ответствующий орган государственной регистрации.
4. ОСОБЫЕ УСЛОВИЯ ДОГОВОРА.
4 1 ОБЪЕКТ должен соответствовать согласованной в установленном порядке проектной документации на ОБЪЕКТ и паспорту его внешнего вида.	испол
4 2 Г1ри переходе гтрава собственности на ОБЪЕКТ на приобретателя права нереходит	чем за
обязанность по возмещению задолженности АРЕНДАТОРА по уплате арендной платы по ДОГОВОРУ, о чем АРЕНДАТОР обязан уведомить приобретателя права,	п.латег
предос
5, АРЕНДНАЯ ПЛАТА.
5 АРЕНДАТОР ежемесячно не позднее 10 (десятого) числа ка;кдого месяца	опреде
уплачивает в бюджет арендную плату, исчисленную АРЕНДОДАТЕЛЕМ в СООТВЕТСТВИИ с решением Тульской городской думы от 22.04.1999г. Уд] 8/297, при этом:	необхс
первый арендный платеж производится до 1 О (десятого) числа следующего за месяцем государственной регистрации ДОГОВОРА и состоит из соответствующей доли арендной платы до последнего числа текущего месяца и платы авансом за последующий месяц; последующие платежи производятся в виде аванса за следующий месяце
5 2 В течение срока действия ДОГОВОРА размер арендной платы, исчисленной на дату заключения ДОГОВОРА, может изменяться в бесспорном порядке, Основанием для бесспорного изменения размера арендной платы являются соответствующие государственные или муниципальные нормативные акты,
5 З. О произведенном изменении арендной платы АРЕНДОДАТЕЛЬ уведомляет	газету
АРЕНДАТОРА.
5 4. АРЕНДАТОР вправе по своему усмоарению производить авансовые платежи до конца срока действия ДОГОВОРА, при этом, если ДОГОВОР был досрочно расторгнут,	норядк€
авансовые платежи возврату не подлежат.	срок с д
5	5 Если после произведённого авансового платежа соответствующим нормативным актом изменилась ставка арендной платы в сторону увеличения, обязан адреса, возместить разницу с момента, указанного в нормативном акте. даты соз 5 б Если после произведённого авансового платежа соответствующим нормативным актом изменилась ставка арендной платы в сторону уменыдения, АРЕНДАТОРУ УЧАСТ засчитывается уплата в счёт будущих платежей, со старь 5 7. В соответствии со сг.4З2 ГК РФ стороны согласны. что существенным условием отдельн} настоящего ДОГОВОРА является обязанность АРЕНДАТОРА ежемесячно предоставлять г Тулы , АРЕНДОДАТЕЛЮ копию платёжного документа о перечислении суммы арендной. платы не позднее 20-го числа текущего месяца (порядок по аналогии с [1.5 1 ДОГОВОРА) заявлени Г]латежный документ, как подтверждение уплаты арендной платы, предоставляется г. Тулы; АРЕНДОДАТЕЛО непосредственно или почтой, или факсимильной связью.
5,8 Пункт 5 7. ДОГОВОРА не распространяется на тот месяц, по которому произведён йривести авансовый платеж	Ври раст 5 9 В случае неуплаты арендной платы более 2-х месяцев подряд (не предоставйения Иному ли подтверждающего уплату платежного поручения более 2-х раз подряд) АРЕНДОДАТЕЛЬ имеет право взыскать её с АРЕНДАТОРА з безакцептном порядке,
	6. ПРАВА И ОБЯЗАННОСТИ АРЕНДОДАТЕЛЯ.	тсутстви
6	АРЕНДОДАТЕЛЬ имеет право досрочно, в одностороннем порядке, с АРЕНДА? соблюдением требований законодательства, предварительно (за десять гугок) письменно, уведомив об этом АРЕНДАТОРА, расторгнуть ДОГОВОР в порядке и случаях, :џнатежейй предусмотренных законодательством, а также в случае нарушения следующих существенных * Становле условий ДОГОВОРА-
 
  6 1 1 ,Использования АРЕНДАТОРОМ УЧАСТКА не в соответствии с разрешенным использованием, определенным п.2.1„ ДОГОВОРХ
6	2 Невыполнения АРЕНДАТОРОМ п.пА.1„ 724 ДОГОВОРА 
6.1	3 Просрочки (неуплаты) АРЕНДАТОРОМ ежемесячной арендной платы более,
чем за два месяца подряд, при условии отсутствия соответствующего разрешения на отсрочку платежей.
Г!одтверждением просрочки (неуплаты) арендной платы является факт не  предостав.ления АРЕНДАТОРОМ АРЕНДОДАТЕЛЮ копии платежного документа в порядке  определённом п,5А7 ДОГОВОРА,
б 4. Лишения АРЕНДАТОРА лицензии на право осуществления деятельности, необходимой для реализации п.2 „ДОГОВОРА 
6.2	АРЕНДОДАТЕЛЬ обязан в месячный срок уведомлять (письменно или через
 
 газету “Тула вечерняя“) АРЕНДАТОРА о следующих изменениях.
 	6.2.1. своего юридического адреса и номера контактного телефона; б 2 2, ставки арендной платы;
 3. банковских реквизутгов счёта, на который поступают арендные платежи. 
7. ПРАВА И ОБЯЗАННОСТИ АРЕНДАТОРА.
7.1 АРЕНДАТОР имеет право з
7	1 в установленном порядке произвести отчуждение ОБЪЕКТА в пользу иных лиц-,
7 2 при отсутствии задолженности по арендным платежам досрочно в одностороннем порядке (письменно уведомив об этом АРЕНДОДАТЕЛЯ) расторгнуть ДОГОВОР.
7.2.АРЕНДАТОР обязан:
7 21 письменно уведомить АРЕНДОДАТЕЛЯ об отчуждении ОБЪЕКТА в ЛО-дневный
срок с даты. совершения 
7 2 2 письменно уведомить АРЕНДОДАТЕЈ!.Я об изменении своего юридического адреса, али номера контактного телефона, Или банковских реквизитов в 10-дневный срок с даты совершённого и.зменения;
7 2 З самостоятельно или по договору с третьими лицами производить уборку   УЧАСТК& а также территории, по периметру прилегающей к нему на расстоянии 2-х метров со стороны земель, не находящихся в пользовании иных лиц, если иное не оговорено отдельным соглашением с уполномоченными администрацией Центрального района Управы глум 
 
  7.2 4 не менее, чем за месяц до окончания срока действия ДОГОВОРА подать заявление о его пролонгации в установленном порядке в администрацию Центрального района г.Ту:љ:,
7.2.5 в течение десяти суток после расторжения ДОГОВОРА снести С)БЪЕКТ и   привести арендованную территорию в первоначальное состояние (данный пункт не действует при расторжении ДОГОВОРА по основаниям перехода прав собственности на ОБЪЕКТ к  иному лицу).
8. ОТВЕТСТВЕННОСТЬ СТОРОН.
 8.1 . В СЛУЧае неуп.лаТЬ1   арендной Б УСТаНOВЛеННЫЙ СРОК И , отсутствия соответствующего разрешения АРЕНДОДАТЕЛЯ на отсрочку •платежа,
 
 
с АРЕНДАТОР уплачивает в бюджет пеню в размере одного процента от суммы недоимки8.2- В случае разового невыполнения 11.5.7 ДОГОВОРА (при отсутствии авансовых  ллатежей), АРЕНДАТОР уплачивает АРЕНДОДАТЕЛЮ штраф в сумме одного,
установленного в соответствии с законодательством РФ, минимального размера оплаты труда,
S В случае невыполнения 7 ДОГОВОРА два раза подряд (пру авансовых платежей). АРЕНДАТОР уплачивает АРЕНДОДАТЕЛО штраф в су установленных в соответствии с законодательством РФ, минимальных размеров оп
 
S,A: В случае невыполнения п, 5 7 ДОГОВОРА. более двух раз подряд (ПРЕ авансовых .адатежей). АРЕНДАТОР уплачивает АРЕНДОДАТЕЛЮ штраф в сумв установленных в соответствии с законодательством РФ, минимальных размеров оп.
S.-S В случае невыполнения п.п.   ДОГОВОРА, АРЕНДАТОР  АРЕНДОДАТЕЈПО штраф в сумме десяти. установленных в соответствии с законо
РФ, минимальных размеров оплаты труда-
S 6 В случае	невыполнения 	ДОГОВОРА, АРЕНДАТОР АРЕНДОДАТЕЛЯ) штраф а сумме 50-ти, установленных в соответствии с законо  РФ, минимальных размеров оплаты труда.
8.7. В случае	невыполнения п-2 1. ДОГОВОРА, АРЕНДАТОР  АРЕНДОДАТЕЛЮ штраф в сумме ста, установленных в соответствии с законода•геј минимальных размеров оплаты труда 
S 8 АРЕНДАТОР освобождается от уплаты штрафа за невыполнение п
ДОГОВОРА, если это невыполнение :лроизотлло в связи с невыполнением АРЕНДС
 ДОГОВОРА 
S 9 Указанные в п.п 8 8 8 ДОГОВОРА суммы неустойки АРЕНДОДАТ  взыскать с АРЕНДАТОРА в безакцептном порядке.
8 1 Г). Уплата штрафа не освобождает АРЕНДАТОРА от надлежащего Е условий ДОГОВОРА 
9. ФОРС-МАЖОРНЫЕ ОБСТОЯТЕЛЬСТВА
9 1 Стороны не несуг ответственности наступлении форс обстоятельств,
9 2 К форс-мажорным обстоятельствам по,мимо общепринятых, относитсз государственными или МУНИЦИП&ЛЪНЫМИ органами власти актов, делающих нев надлежащее исполнение ДОГОВОРА 
93 При возникновении форс-мажорных обстоятельств ДОГОВОР расторгнутым по письменному уведомлению о расторжении одной из сторон.
10. РАССМОТРЕНИЕ СПОРОВ,
10.1. Споры. возникшие из реализации ДОГОВОРА, которые не удалось путём переговоров, разрешаются в Арбитражном суде Тульской области. 
П. ИЗМЕНЕНИЕ И ДОПОЛНЕНИЕ ДОГОВОР А
1 1. Условия настоящего ДОГОВОРА могут быть изменены и дополнены т согласии сторон.
12. ЗАКЛЮЧИТЕЛЬНЫЕ ПОЛОЖЕНИЯ
12.1 ДОГОВОР составлен в трёх экземплярах на трех листах на русском я экземпляры идентичны и имеют одинаковую силу. У каждой из сторон находится г экземпляру ДОГОВОРА, третий экземпляр находится в орга}Ае государственной регис
 
12,2Неотъемлемой частые договора является план земельного удостоверенный Комитетом по земельным ресурсам землеусаройсгву г- Тулы. У к сторон находится по одному экземпляру плана, третий экземпляр находится  государственной регистрации
  отсутствий гмме десяти\ златы труда.             и отсутствий	12 З, ДОГОВОР вступает в силу с момента его государственной регистрации,
ме тридцату латы труда-          	ЮРИДИЧЕСКИЕ АДРЕСА СТОРОН:
д уплачивает )дательствоу	АРЕНДОДАТЕЛЬ -300034» г, Тула, ул, Гоголевская, д. 7.3 
уплачива )дателъство	ПОДПИСИ СТОРОН:
уплачива эльством РФ
65,1 и п.5.
РДАТЕЛЕУ
ТЕЛЬ вправ
выполнени
)с-мажорнь
  приняти евозможны
 считаетс
ъ разрешить
только пр
ЯЗЫКе. Все по оддому гистрациид участка, каждой из я в органе

"""

text += """
МИНИСТЕРСТВО
ИМУЩЕСТВЕННЫХ И
ЗЕМЕЛЬНЫХ ОТНОШЕНИЙ
ТУЛЬСКОЙ ОБЛАСТИ
ул. Жаворонкова, д.2, г. Тула, 300012 тел.: (4872) 24-53-33, факс: 36-14-11
E-mail: mizo@tularegion.ru  htt :i/mizo. tularegion.ru
ПРЕТЕНЗИЯ
В соответствии с договором аренды земельного участка NQO
(далее -: Договор) был предоставлен в аренду земельный участок с кадастровым номером для организации входа в выставочный зал, расположенного по адресу:
Договор действует.
В соответствии с условиями Договора арендаторы обязаны своевременно вносить арендную плату за пользование земельным участком.
В нарушение условий Договора за  числится задолженность по арендной плате в размере— руб. и по пени в размере руб. за период с
 
Исходя из вышеизложенного, ООО «Стикс» надлежит: в 10-дневный срок со дня получения претензии оплатить образовавшуюся задолженность.
Реквизиты для оплаты:
Получатель: УФК по Тульской области (Мин истперство имущественных и земельных отношений Тульской области), ИНН: 7106058814, КПП: 710601001,Расчетный счет: 03100643000000016600, КОРСЧЕТ: 40102810445370000059, Банк получателя: ОТДЕЛЕНИЕ ТУЛА БАНКА РОССИИ//УФК по тульской области г. Тула, БИК: 017003983, код ОКТМО: 70701000, коды бюджепшой классификации: 82311105012040000120 -аренДная планш по Договору аренды земельного учаспжа; 82311607090040000140 — пени по Договору аренды земельного учаспжа; Назначение платежа: аренДная плата за земельный участок по Договору аренДы земельного участка 1фот  НДС не облагается (заполнение назначен ия платежа обязапгельно). Пени уплачиваются   плапгеЖНЬIМ пор чениелт. Назначение платежа: пени по Договору аренды земельного участка N от  ндс не облагается (заполнен ие назначения платежа обязательно).
В случае отказа или неполучения ответа в 10-дневный срок, министерство имущественных и земельных отношений Тульской области обратиться в суд с заявлением о взыскании указанной задолженности в судебном порядке с отнесением  расходов по уплате госпошлины.
Заместитель министра и мущественных и земельных отношений Тульской области  
тел. 

"""
contract_vector = vectorizer.transform([text])

In [10]:
contract_prediction = model.predict(contract_vector)
# Арбитражный суд: 0 - да, 1 - нет
print(f"Предсказание для договора: {contract_prediction[0]}")


1/1 [==============================] - 0s 11ms/step
Предсказание для договора: [0.0001034]
